## Import

In [1]:
import os, io, gc, glob
import numpy as np
import pandas as pd
import random

from scipy.fft import fft
from scipy.signal import hilbert, blackman
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten ,Dropout, Activation
from tensorflow.keras.layers import Add, concatenate
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings('ignore')
gc.enable()

## Fix Seed

In [2]:
SEED = 42
def seed_everything(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
seed_everything()

## Hyperparameter Setting

In [3]:
# Model Configuration
class CFG:
    seed = 42
    VERBOSE = 1
    random_state = 42
    N_FOLDS = 28
    EPOCHS = 200
    BATCH_SIZE = 8
    factor = 0.5
    patience_1 = 5
    patience_2 = 15
    learning_rate = 1e-3
    weight_decay = 1e-3
    dropout_rate = 0.2

## Data loading

In [4]:
all_input_list = sorted(glob.glob('./data/train_input/*.csv'))
all_target_list = sorted(glob.glob('./data/train_target/*.csv'))

df_train_input = pd.concat(map(pd.read_csv, all_input_list)).drop(columns='obs_time')
df_train_target = pd.concat(map(pd.read_csv, all_target_list)).drop(columns='DAT')

train_input = df_train_input.to_numpy().reshape(-1,24,15)
train_target = df_train_target.to_numpy()

## Model define

In [5]:
def dnn_model():
    x_input = Input(shape=[24,15])
    
    x1 = Bidirectional(LSTM(units=1024, return_sequences=True))(x_input)
    c1 = concatenate([x_input, x1])
    
    x2 = Bidirectional(LSTM(units=512, return_sequences=True))(c1)
    c2 = concatenate([x1, x2])
    
    x3 = Bidirectional(LSTM(units=256, return_sequences=True))(c2)
    c3 = concatenate([x2, x3])
    
    x4 = Bidirectional(LSTM(units=128, return_sequences=True))(c3)
    c4 = concatenate([x3, x4])
    
    x5 = Dense(units=128, activation='selu')(c4)
    x6 = Flatten()(x5)
    x_output = Dense(units=1)(x6)
    
    model = Model(inputs=x_input, outputs=x_output, name='DNN_Model')
    model.compile(optimizer='Adam', loss='mae')
    
    return model

In [6]:
test_preds = []
history_list = []
oof_true = []
oof_pred = []

## KFold cross-validation

In [7]:
kf = KFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=42)
oof_preds = np.zeros((train_input.shape[0],train_input.shape[1]))
history_list = []

# Scaler
MMS_input,MMS_target = MinMaxScaler(feature_range=(0.0,1.0)),MinMaxScaler(feature_range=(0.0,1.0))

for fold, (train_idx, test_idx) in enumerate(kf.split(train_input, train_target)):
    print('='*25, '>', f'Fold {fold+1}', '<', '='*25)
    checkpoint_filepath = f'fold{fold+1}.hdf5'

    X_train, X_valid = train_input[train_idx], train_input[test_idx]
    y_train, y_valid = train_target[train_idx], train_target[test_idx]
    
    # data scaling
    X_train = X_train.reshape(-1,15)
    X_valid = X_valid.reshape(-1,15)
    X_train = MMS_input.fit_transform(X_train)
    y_train = MMS_target.fit_transform(y_train)
    X_valid = MMS_input.transform(X_valid)
    y_valid = MMS_target.transform(y_valid)
    X_train = X_train.reshape(-1,24,15)
    X_valid = X_valid.reshape(-1,24,15)
        
    lr = ReduceLROnPlateau(monitor="val_loss", factor=CFG.factor, patience=CFG.patience_1,
                               verbose=CFG.VERBOSE)
    es = EarlyStopping(monitor="val_loss", patience=CFG.patience_2, mode="min",
                           restore_best_weights=True, verbose=CFG.VERBOSE)
    sv = ModelCheckpoint(checkpoint_filepath, monitor = 'val_loss', verbose = CFG.VERBOSE,
                             save_best_only = True, save_weights_only = True, mode = 'min')
        
    model = dnn_model()
    history = model.fit(X_train,y_train,
                        validation_data=(X_valid,y_valid),
                        epochs=CFG.EPOCHS, batch_size=CFG.BATCH_SIZE, callbacks=[lr,es,sv])        
    history_list += [history]
        
    # predict oof
    y_pred = model.predict(X_valid)
    y_true = y_valid
        
    ## inspiratory and expiratory phases
    score = mean_absolute_error(y_true, y_pred)
    print(f'Fold {fold+1} | Overall MAE Score: {score}')
        
    # predict test
    test_preds.append(model.predict(X_valid))
    del X_train, X_valid, y_train, y_valid
    gc.collect()
        
np.save('test_preds.npy', test_preds)

========================= > Fold 1 < =========================
Epoch 1/200
95/95 [==============================] - 12s 52ms/step - loss: 0.2602 - val_loss: 0.1489

Epoch 00001: val_loss improved from inf to 0.14887, saving model to fold1.hdf5
Epoch 2/200
95/95 [==============================] - 4s 37ms/step - loss: 0.1147 - val_loss: 0.1213

Epoch 00002: val_loss improved from 0.14887 to 0.12131, saving model to fold1.hdf5
Epoch 3/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0997 - val_loss: 0.0703

Epoch 00003: val_loss improved from 0.12131 to 0.07029, saving model to fold1.hdf5
Epoch 4/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0933 - val_loss: 0.0743

Epoch 00004: val_loss did not improve from 0.07029
Epoch 5/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0795 - val_loss: 0.1346

Epoch 00005: val_loss did not improve from 0.07029
Epoch 6/200
95/95 [==============================] - 3s 36ms/step - loss: 0.1081 - v

95/95 [==============================] - 3s 36ms/step - loss: 0.0251 - val_loss: 0.0293

Epoch 00049: val_loss did not improve from 0.02497
Epoch 50/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0233 - val_loss: 0.0269

Epoch 00050: val_loss did not improve from 0.02497
Epoch 51/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0221 - val_loss: 0.0264

Epoch 00051: val_loss did not improve from 0.02497
Epoch 52/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0237 - val_loss: 0.0268

Epoch 00052: val_loss did not improve from 0.02497
Epoch 53/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0222 - val_loss: 0.0279

Epoch 00053: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00053: val_loss did not improve from 0.02497
Epoch 54/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0213 - val_loss: 0.0279

Epoch 00054: val_loss did not improve from 0.02497
Epoch 55/200
95

95/95 [==============================] - 3s 36ms/step - loss: 0.0120 - val_loss: 0.0223

Epoch 00098: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 00098: val_loss did not improve from 0.02201
Epoch 99/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0119 - val_loss: 0.0223

Epoch 00099: val_loss did not improve from 0.02201
Epoch 100/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0119 - val_loss: 0.0223

Epoch 00100: val_loss did not improve from 0.02201
Epoch 101/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0119 - val_loss: 0.0223

Epoch 00101: val_loss did not improve from 0.02201
Epoch 102/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0119 - val_loss: 0.0221

Epoch 00102: val_loss did not improve from 0.02201
Epoch 103/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0119 - val_loss: 0.0222

Epoch 00103: ReduceLROnPlateau reducing learning rate to 4.88

95/95 [==============================] - 3s 36ms/step - loss: 0.0251 - val_loss: 0.0478

Epoch 00041: val_loss did not improve from 0.02377
Epoch 42/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0245 - val_loss: 0.0312

Epoch 00042: val_loss did not improve from 0.02377
Epoch 43/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0239 - val_loss: 0.0288

Epoch 00043: val_loss did not improve from 0.02377
Epoch 44/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0248 - val_loss: 0.0354

Epoch 00044: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00044: val_loss did not improve from 0.02377
Epoch 45/200
95/95 [==============================] - 3s 37ms/step - loss: 0.0214 - val_loss: 0.0295

Epoch 00045: val_loss did not improve from 0.02377
Epoch 46/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0216 - val_loss: 0.0303

Epoch 00046: val_loss did not improve from 0.02377
Epoch 47/200
95

Epoch 40/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0284 - val_loss: 0.0340

Epoch 00040: val_loss did not improve from 0.02891
Epoch 41/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0297 - val_loss: 0.0345

Epoch 00041: val_loss did not improve from 0.02891
Epoch 42/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0283 - val_loss: 0.0353

Epoch 00042: val_loss did not improve from 0.02891
Epoch 43/200
95/95 [==============================] - 3s 37ms/step - loss: 0.0283 - val_loss: 0.0393

Epoch 00043: val_loss did not improve from 0.02891
Epoch 44/200
95/95 [==============================] - 3s 37ms/step - loss: 0.0307 - val_loss: 0.0533

Epoch 00044: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00044: val_loss did not improve from 0.02891
Epoch 45/200
95/95 [==============================] - 3s 36ms/step - loss: 0.0267 - val_loss: 0.0356

Epoch 00045: val_loss did not improve from 0.02891
E

...

95/95 [==============================] - 3s 34ms/step - loss: 0.0501 - val_loss: 0.0353

Epoch 00022: val_loss did not improve from 0.02998
Epoch 23/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0488 - val_loss: 0.0383

Epoch 00023: val_loss did not improve from 0.02998
Epoch 24/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0510 - val_loss: 0.0478

Epoch 00024: val_loss did not improve from 0.02998
Epoch 25/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0517 - val_loss: 0.0261

Epoch 00025: val_loss improved from 0.02998 to 0.02608, saving model to fold13.hdf5
Epoch 26/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0489 - val_loss: 0.0385

Epoch 00026: val_loss did not improve from 0.02608
Epoch 27/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0489 - val_loss: 0.0394

Epoch 00027: val_loss did not improve from 0.02608
Epoch 28/200
95/95 [==============================] - 3s 34ms/

95/95 [==============================] - 3s 34ms/step - loss: 0.0142 - val_loss: 0.0231

Epoch 00071: val_loss did not improve from 0.01862
Epoch 72/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0136 - val_loss: 0.0232

Epoch 00072: val_loss did not improve from 0.01862
Epoch 73/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0138 - val_loss: 0.0209

Epoch 00073: val_loss did not improve from 0.01862
Epoch 74/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0138 - val_loss: 0.0224

Epoch 00074: val_loss did not improve from 0.01862
Epoch 75/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0136 - val_loss: 0.0215

Epoch 00075: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Restoring model weights from the end of the best epoch.

Epoch 00075: val_loss did not improve from 0.01862
Epoch 00075: early stopping
Fold 13 | Overall MAE Score: 0.018616763451156964
========================= > Fold 

95/95 [==============================] - 3s 34ms/step - loss: 0.0234 - val_loss: 0.0281

Epoch 00044: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00044: val_loss did not improve from 0.02577
Epoch 45/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0219 - val_loss: 0.0258

Epoch 00045: val_loss did not improve from 0.02577
Epoch 46/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0205 - val_loss: 0.0194

Epoch 00046: val_loss improved from 0.02577 to 0.01943, saving model to fold14.hdf5
Epoch 47/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0208 - val_loss: 0.0248

Epoch 00047: val_loss did not improve from 0.01943
Epoch 48/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0200 - val_loss: 0.0241

Epoch 00048: val_loss did not improve from 0.01943
Epoch 49/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0200 - val_loss: 0.0266

Epoch 00049: val_loss did not imp

95/95 [==============================] - 3s 34ms/step - loss: 0.0652 - val_loss: 0.0664

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00010: val_loss did not improve from 0.06131
Epoch 11/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0525 - val_loss: 0.0414

Epoch 00011: val_loss improved from 0.06131 to 0.04143, saving model to fold15.hdf5
Epoch 12/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0523 - val_loss: 0.0303

Epoch 00012: val_loss improved from 0.04143 to 0.03030, saving model to fold15.hdf5
Epoch 13/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0481 - val_loss: 0.0594

Epoch 00013: val_loss did not improve from 0.03030
Epoch 14/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0475 - val_loss: 0.0404

Epoch 00014: val_loss did not improve from 0.03030
Epoch 15/200
95/95 [==============================] - 3s 34ms/step - loss: 0.0529 - val_loss: 0.0297


ResourceExhaustedError:    OOM when allocating tensor with shape[1024,15] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node transpose_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[DNN_Model/bidirectional_60/forward_lstm_60/PartitionedCall]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_595298]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
oof_preds = oof_preds.squeeze().reshape(-1,1).squeeze()
reshaped_targets = targets.squeeze().reshape(-1,1).squeeze()
score = mean_absolute_error(reshaped_targets, oof_preds)
print(f'Overall OOF MAE Score: {score}')

In [ ]:
def plot_hist(hist, with_grid=True):
    plt.figure(figsize=(20,5))
    for i in range(len(hist)):
        plt.plot(hist[i].history["loss"], color='grey')
        plt.plot(hist[i].history["val_loss"], color='green')
    plt.title("")
    plt.ylabel("Mean Absolute Error")
    plt.xlabel("epoch")
    plt.legend(["Training", "Validation"], loc="upper right")
    if with_grid:
        plt.grid(which='major', axis='both')
    plt.show()
plot_hist(history_list)